## Deliverable 2. Create a Customer Travel Destinations Map.

In [17]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lorengau,PG,-2.0226,147.2712,81.88,78,99,7.07,overcast clouds
1,1,Ayorou,NE,14.7308,0.9174,71.71,13,3,11.05,clear sky
2,2,Dudinka,RU,69.4058,86.1778,-3.39,94,100,13.87,overcast clouds
3,3,Nueve De Julio,AR,-35.4444,-60.8831,82.71,60,38,5.82,scattered clouds
4,4,Jamestown,US,42.0970,-79.2353,12.09,54,0,3.00,clear sky


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lorengau,PG,-2.0226,147.2712,81.88,78,99,7.07,overcast clouds
1,1,Ayorou,NE,14.7308,0.9174,71.71,13,3,11.05,clear sky
3,3,Nueve De Julio,AR,-35.4444,-60.8831,82.71,60,38,5.82,scattered clouds
8,8,Kapaa,US,22.0752,-159.3190,80.58,67,0,3.00,clear sky
16,16,Salinopolis,BR,-0.6136,-47.3561,75.43,89,100,5.23,overcast clouds
19,19,Puerto Ayora,EC,-0.7393,-90.3518,80.56,72,31,8.25,scattered clouds
21,21,Puerto Escondido,MX,15.8500,-97.0667,83.23,65,20,9.22,few clouds
23,23,Tambopata,PE,-12.7333,-69.1833,74.64,97,97,1.54,overcast clouds
28,28,Port Alfred,ZA,-33.5906,26.8910,70.12,85,44,12.08,scattered clouds
30,30,Sao Miguel Do Araguaia,BR,-13.2750,-50.1628,72.23,93,99,6.04,overcast clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Lorengau,PG,81.88,overcast clouds,-2.0226,147.2712,
1,Ayorou,NE,71.71,clear sky,14.7308,0.9174,
3,Nueve De Julio,AR,82.71,scattered clouds,-35.4444,-60.8831,
8,Kapaa,US,80.58,clear sky,22.0752,-159.3190,
16,Salinopolis,BR,75.43,overcast clouds,-0.6136,-47.3561,
19,Puerto Ayora,EC,80.56,scattered clouds,-0.7393,-90.3518,
21,Puerto Escondido,MX,83.23,few clouds,15.8500,-97.0667,
23,Tambopata,PE,74.64,overcast clouds,-12.7333,-69.1833,
28,Port Alfred,ZA,70.12,scattered clouds,-33.5906,26.8910,
30,Sao Miguel Do Araguaia,BR,72.23,overcast clouds,-13.2750,-50.1628,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.NaN, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Lorengau,PG,81.88,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
1,Ayorou,NE,71.71,clear sky,14.7308,0.9174,Hotel Amenokal
3,Nueve De Julio,AR,82.71,scattered clouds,-35.4444,-60.8831,Grand Hotel Libertad - GHL
8,Kapaa,US,80.58,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
16,Salinopolis,BR,75.43,overcast clouds,-0.6136,-47.3561,Salinas Park Resort Maçarico AP101


In [27]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))